# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output_data/cities.csv")
city_data

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,0,Busselton,AU,1621740615,-33.6500,115.3333,85,70,57.04,17.78
1,1,Killybegs,IE,1621740615,54.6333,-8.4500,100,91,43.45,21.05
2,2,Longyearbyen,SJ,1621740616,78.2186,15.6401,20,58,24.64,3.44
3,3,Mataura,NZ,1621740616,-46.1927,168.8643,3,79,49.48,0.69
4,4,Distrikt Brokopondo,SR,1621740617,4.7500,-55.1667,96,91,76.53,7.07
...,...,...,...,...,...,...,...,...,...,...
497,498,Ust-Maya,RU,1621740836,60.4167,134.5333,74,24,56.34,4.00
498,499,Husavik,IS,1621740837,66.0449,-17.3389,100,96,40.91,9.86
499,500,Ano Poroia,GR,1621740837,41.2833,23.0333,70,73,60.24,2.66
500,501,Narsaq,GL,1621740838,60.9167,-46.0500,100,68,35.82,5.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [7]:
locations = city_data[["Latitude", "Longitude"]].astype(float)
humidity = city_data["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
narrowed_city_df = city_data.loc[(city_data["Max Temperature"] < 80) & (city_data["Max Temperature"] > 70) \
                                    & (city_data["Wind Speed"] < 10) \
                                    & (city_data["Cloudiness"] == )].dropna()
narrowed_city_df

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
176,177,Kibala,TD,1621740697,9.1104,18.3463,0,83,78.75,9.22
214,215,Adrar,MR,1621740716,20.5022,-10.0711,1,23,83.88,3.24
263,264,Mastic Beach,US,1621740738,40.7668,-72.8521,1,75,77.25,3.00
337,338,Dhārchula,IN,1621740770,29.8500,80.5333,0,45,79.59,6.06
363,364,Bethel,US,1621740781,41.3712,-73.4140,1,78,75.78,0.00
406,407,Saint Anthony,US,1621740799,45.0205,-93.2180,1,79,76.62,1.01
438,439,Humberto de Campos,BR,1621740812,-2.5983,-43.4611,3,94,76.19,4.23
478,479,Griffith,US,1621740828,41.5284,-87.4237,1,64,77.09,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = narrowed_city_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
176,Kibala,TD,9.1104,18.3463,
214,Adrar,MR,20.5022,-10.0711,
263,Mastic Beach,US,40.7668,-72.8521,
337,Dhārchula,IN,29.8500,80.5333,
363,Bethel,US,41.3712,-73.4140,
406,Saint Anthony,US,45.0205,-93.2180,
438,Humberto de Campos,BR,-2.5983,-43.4611,
478,Griffith,US,41.5284,-87.4237,


In [47]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Latitude,Longitude,Hotel Name
176,Kibala,TD,9.1104,18.3463,
214,Adrar,MR,20.5022,-10.0711,
263,Mastic Beach,US,40.7668,-72.8521,
337,Dhārchula,IN,29.8500,80.5333,
363,Bethel,US,41.3712,-73.4140,
406,Saint Anthony,US,45.0205,-93.2180,
438,Humberto de Campos,BR,-2.5983,-43.4611,
478,Griffith,US,41.5284,-87.4237,


In [48]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [49]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))